In [1]:
import pandas as pd

In [8]:
df = pd.read_csv("spotify_millsongdata.csv")

In [9]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [10]:
df.shape

(57650, 4)

In [11]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [12]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [13]:
df.head()

,artist,song,text
0,Eric Clapton,Hold On I'm Coming,"Well, don't you ever be sad \r\nLean on me wh..."
1,Peter Gabriel,Don't Break This Rhythm,"Don't break this rhythm, don't break this moti..."
2,Westlife,Love Crime,I didn't know what to say \r\nI tried to look...
3,Lauryn Hill,When It Hurts So Bad,When it hurts so bad (when it hurts so bad) \...
4,Boney M.,Bel Ami,He's a casanova (yeah yeah what'd I say?) \r\...


In [14]:
df['text'][0]

"Well, don't you ever be sad  \r\nLean on me when times are bad  \r\nWhen the day comes and you're in doubt  \r\nIn a river of trouble, about to drown  \r\n  \r\nHold on, hold on, I'm coming, I'm coming  \r\nHold on, hold on, I'm coming, I'm coming  \r\n  \r\nI'm on my way, your lover  \r\nWhen you get cold, yeah, I'll be your cover  \r\nDon't have to worry because I'm here  \r\nDon't need to suffer because I'm here  \r\n  \r\nHold on, hold on, I'm coming, I'm coming  \r\nHold on, hold on, I'm coming, I'm coming  \r\nHold on, hold on, I'm coming, I'm coming  \r\nHold on, hold on, I'm coming, I'm coming  \r\n  \r\nReach out to me for satisfaction  \r\nCall my name, quick reaction  \r\n  \r\nWell, don't you ever feel sad  \r\nLean on me when times are bad  \r\nWhen the day comes and you're in doubt  \r\nIn a river of trouble and about to drown  \r\n  \r\nHold on, hold on, I'm coming, I'm coming  \r\nHold on, hold on, I'm coming, I'm coming  \r\nHold on, hold on, I'm coming, I'm coming  \

In [15]:
#df = df.sample(5000)

In [16]:
df.shape

(5000, 3)

# Text Cleaning/ Text Preprocessing


In [17]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [43]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [44]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [47]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [48]:
similarity[0]

array([1.        , 0.0275174 , 0.02964029, ..., 0.01921103, 0.02715013,
       0.00476966])

In [66]:
df[df['song']=='Waiting For The Man']

,artist,song,text


In [68]:
filtered_df = df[df['song'] == 'Waiting For The Man'].index[0]

# Display the filtered DataFrame
print(filtered_df)

82


In [69]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [70]:
recommendation('Waiting For The Man')

['Gone Too Long',
 'Love Me Tonight',
 'Long Gone And Moved On',
 "My Baby's Gone",
 'Get Down, Make Love',
 "Burnin' Up",
 'I Just Want To Make Love To You',
 "I'll Love You For A While",
 'I Love Making Love To You',
 'Are You Gonna Love Me',
 'No One Can Love You More',
 'You Give Me Love',
 'Loving You Tonight',
 'Baby I Love You',
 'Love Remains',
 'The Goods Gone',
 'Live It Up Tonight',
 'Who Do You Love',
 'Lost In Love',
 'Gone With The Wind']

In [71]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))